# 1. 라이브러리 임포트

In [1]:
### import
import pandas as pd
import numpy as np

# 데이터 프레임 row 제한
pd.set_option('display.max_columns', 60)

# matplotlib 화면 표시
import matplotlib.pyplot as plt
%matplotlib inline

# 폰트사이즈 설정
plt.rcParams['font.size'] = 24

# Internal ipython tool for setting figure size
from IPython.core.pylabtools import figsize

# Seaborn 화면 표시
import seaborn as sns
sns.set(font_scale = 2)

# 데이터 분리를 통한 훈련데이터와 테스터 데이터 학습
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [2]:
# 저장한 모델 불러오기
from numpy import loadtxt
import xgboost
import pickle
from sklearn import model_selection
from sklearn.metrics import accuracy_score
#pickle.dump(final_model, open("pima.pickle.dat", "wb"))
kj_model = pickle.load(open("pima.pickle.dat", "rb"))

# 2. 테스트 데이터 예측

In [3]:
temp = pd.read_csv("dataset/credit_temp.csv")
temp

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term_Long Term,Term_Short Term,Years in current job_1 year,Years in current job_10+ years,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Home Ownership_Rent,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
test = pd.read_csv('dataset/credit_test.csv')
test

,Loan ID,Customer ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,f738779f-c726-40dc-92cf-689d73af533d,ded0b3c3-6bf4-4091-8726-47039f2c1b90,611314.0,Short Term,747.0,2074116.0,10+ years,Home Mortgage,Debt Consolidation,42000.83,21.8,NaN,9.0,0.0,621908.0,1058970.0,0.0,0.0
1,6dcc0947-164d-476c-a1de-3ae7283dde0a,1630e6e3-34e3-461a-8fda-09297d3140c8,266662.0,Short Term,734.0,1919190.0,10+ years,Home Mortgage,Debt Consolidation,36624.40,19.4,NaN,11.0,0.0,679573.0,904442.0,0.0,0.0
2,f7744d01-894b-49c3-8777-fc6431a2cff1,2c60938b-ad2b-4702-804d-eeca43949c52,153494.0,Short Term,709.0,871112.0,2 years,Rent,Debt Consolidation,8391.73,12.5,10.0,10.0,0.0,38532.0,388036.0,0.0,0.0
3,83721ffb-b99a-4a0f-aea5-ef472a138b41,12116614-2f3c-4d16-ad34-d92883718806,176242.0,Short Term,727.0,780083.0,10+ years,Rent,Debt Consolidation,16771.87,16.5,27.0,16.0,1.0,156940.0,531322.0,1.0,0.0
4,08f3789f-5714-4b10-929d-e1527ab5e5a3,39888105-fd5f-4023-860a-30a3e6f5ccb7,321992.0,Short Term,744.0,1761148.0,10+ years,Home Mortgage,Debt Consolidation,39478.77,26.0,44.0,14.0,0.0,359765.0,468072.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# 데이터 더미함수화
def callData(data):
    del data["Loan ID"]
    del data["Customer ID"]
    del data["Months since last delinquent"]
    data = data.dropna()
    # 범주형 데이터 확인
    categorical_subset = data[['Term', 'Years in current job', 'Home Ownership', 'Purpose']]

    # one hot encoding(더미함수 사용)
    categorical_subset = pd.get_dummies(categorical_subset)

    data.drop(labels=['Term', 'Years in current job', 'Home Ownership', 'Purpose'], axis=1, inplace=True)
    data = pd.concat([data, categorical_subset], axis = 1)
    data = pd.concat([temp,data])
    data = data.fillna(0)
    data = data[1:]
    return data


def scailData(data):
    
    data = sc.fit_transform(data)
    
    return data

In [6]:
# 칼럼데이터 가져오기
#temp = pd.read_csv("dataset/credit_temp.csv")
# 테스트 데이터 불러오기
#test.head()
test = callData(test)
test

C:\ProgramData\Anaconda3\envs\juno\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term_Long Term,Term_Short Term,Years in current job_1 year,Years in current job_10+ years,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Home Ownership_Rent,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,611314.0,747.0,2074116.0,42000.83,21.8,9.0,0.0,621908.0,1058970.0,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,266662.0,734.0,1919190.0,36624.40,19.4,11.0,0.0,679573.0,904442.0,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,153494.0,709.0,871112.0,8391.73,12.5,10.0,0.0,38532.0,388036.0,0.0,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,176242.0,727.0,780083.0,16771.87,16.5,16.0,1.0,156940.0,531322.0,1.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,321992.0,744.0,1761148.0,39478.77,26.0,14.0,0.0,359765.0,468072.0,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,442596.0,739.0,1528968.0,19494.38,16.7,11.0,0.0,419235.0,656876.0,0.0,0.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9995,157806.0,731.0,1514376.0,4795.41,12.5,9.0,0.0,87058.0,234410.0,0.0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9996,132550.0,718.0,763192.0,12401.87,9.9,8.0,0.0,74309.0,329692.0,0.0,0.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9998,99999999.0,721.0,972097.0,12232.20,16.8,8.0,1.0,184984.0,240658.0,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
test_pury = test
test_pury = sc.fit_transform(test_pury)

In [8]:
test_pury

array([[-0.39730937, -0.2247615 ,  0.78549487, ..., -0.05836702,
        -0.0228605 , -0.03233808],
       [-0.40723134, -0.23354045,  0.60823869, ..., -0.05836702,
        -0.0228605 , -0.03233808],
       [-0.41048927, -0.25042306, -0.59090354, ..., -0.05836702,
        -0.0228605 , -0.03233808],
       ...,
       [-0.41109221, -0.24434532, -0.71437854, ..., -0.05836702,
        -0.0228605 , -0.03233808],
       [ 2.46392947, -0.24231941, -0.47536312, ..., -0.05836702,
        -0.0228605 , -0.03233808],
       [ 2.46392947, -0.22408619, -0.35195333, ..., -0.05836702,
        -0.0228605 , -0.03233808]])

In [9]:
#y_pred = final_model.predict(test_pury[27:28])
y_pred = kj_model.predict(test_pury)
# y_pred = list(map(lambda x: 'Fully Paid' if x==1 else 'Charged Off' ,y_pred))
# y_pred = np.array(y_pred)
# y_pred
#np.where(y_pred==0)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

# 3. 한명의 고객 데이터만 예측

In [10]:
test_new = pd.read_csv('dataset/train_new.csv')
test_new

,Loan ID,Customer ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,f8fa11bd-d0d2-4c9f-998e-9f9d866803a7,ab86ebb2-c4f4-47dd-b430-3d6270394f8b,468908,Long Term,6470,1833842,8 years,Rent,other,5898.93,14.4,NaN,19,1,183863,809490,1,0


In [11]:
test

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term_Long Term,Term_Short Term,Years in current job_1 year,Years in current job_10+ years,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Home Ownership_Rent,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,611314.0,747.0,2074116.0,42000.83,21.8,9.0,0.0,621908.0,1058970.0,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,266662.0,734.0,1919190.0,36624.40,19.4,11.0,0.0,679573.0,904442.0,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,153494.0,709.0,871112.0,8391.73,12.5,10.0,0.0,38532.0,388036.0,0.0,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,176242.0,727.0,780083.0,16771.87,16.5,16.0,1.0,156940.0,531322.0,1.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,321992.0,744.0,1761148.0,39478.77,26.0,14.0,0.0,359765.0,468072.0,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,442596.0,739.0,1528968.0,19494.38,16.7,11.0,0.0,419235.0,656876.0,0.0,0.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9995,157806.0,731.0,1514376.0,4795.41,12.5,9.0,0.0,87058.0,234410.0,0.0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9996,132550.0,718.0,763192.0,12401.87,9.9,8.0,0.0,74309.0,329692.0,0.0,0.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9998,99999999.0,721.0,972097.0,12232.20,16.8,8.0,1.0,184984.0,240658.0,0.0,0.0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
def callDataone(data):
    del data["Loan ID"]
    del data["Customer ID"]
    del data["Months since last delinquent"]
    data = data.dropna()
    # 범주형 데이터 확인
    categorical_subset = data[['Term', 'Years in current job', 'Home Ownership', 'Purpose']]

    # one hot encoding(더미함수 사용)
    categorical_subset = pd.get_dummies(categorical_subset)

    data.drop(labels=['Term', 'Years in current job', 'Home Ownership', 'Purpose'], axis=1, inplace=True)
    data = pd.concat([data, categorical_subset], axis = 1)
    data = pd.concat([data,test],sort=False)
    #data = pd.concat([temp,data],sort=False)
    data = data.fillna(0)
    #data = data[1:]
    return data

In [13]:
test_new_one = callDataone(test_new)
test_new_one

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term_Long Term,Years in current job_8 years,Home Ownership_Rent,Purpose_other,Term_Short Term,Years in current job_1 year,Years in current job_10+ years,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
0,468908.0,6470.0,1833842.0,5898.93,14.4,19.0,1.0,183863.0,809490.0,1.0,0.0,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,611314.0,747.0,2074116.0,42000.83,21.8,9.0,0.0,621908.0,1058970.0,0.0,0.0,0,0,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,266662.0,734.0,1919190.0,36624.40,19.4,11.0,0.0,679573.0,904442.0,0.0,0.0,0,0,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,153494.0,709.0,871112.0,8391.73,12.5,10.0,0.0,38532.0,388036.0,0.0,0.0,0,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,176242.0,727.0,780083.0,16771.87,16.5,16.0,1.0,156940.0,531322.0,1.0,0.0,0,0,1,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,442596.0,739.0,1528968.0,19494.38,16.7,11.0,0.0,419235.0,656876.0,0.0,0.0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9995,157806.0,731.0,1514376.0,4795.41,12.5,9.0,0.0,87058.0,234410.0,0.0,0.0,0,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,132550.0,718.0,763192.0,12401.87,9.9,8.0,0.0,74309.0,329692.0,0.0,0.0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,99999999.0,721.0,972097.0,12232.20,16.8,8.0,1.0,184984.0,240658.0,0.0,0.0,0,0,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
test[27:28]

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Term_Long Term,Term_Short Term,Years in current job_1 year,Years in current job_10+ years,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Home Ownership_Rent,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
35,468908.0,6470.0,1833842.0,5898.93,14.4,19.0,1.0,183863.0,809490.0,1.0,0.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [15]:
test_puri_one = sc.fit_transform(test_new_one)
test_puri_one

array([[-0.40137858,  3.6366228 ,  0.51054707, ..., -0.05836319,
        -0.02285901, -0.03233597],
       [-0.39727872, -0.22505688,  0.78546612, ..., -0.05836319,
        -0.02285901, -0.03233597],
       [-0.40720123, -0.23382882,  0.60820138, ..., -0.05836319,
        -0.02285901, -0.03233597],
       ...,
       [-0.41106231, -0.24462506, -0.71447969, ..., -0.05836319,
        -0.02285901, -0.03233597],
       [ 2.46411683, -0.24260077, -0.47545273, ..., -0.05836319,
        -0.02285901, -0.03233597],
       [ 2.46411683, -0.22438211, -0.35203699, ..., -0.05836319,
        -0.02285901, -0.03233597]])

In [18]:
#y_pred = final_model.predict(test_pury[27:28])
y_pred = kj_model.predict(test_puri_one[0:1])
# y_pred = list(map(lambda x: 'Fully Paid' if x==1 else 'Charged Off' ,y_pred))
# y_pred = np.array(y_pred)
# y_pred
#np.where(y_pred==0)
y_pred

array([0])